In [1]:
import numpy as np
import os
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.regularizers import l1
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

import matplotlib.pyplot as plt
import pandas as pd
from keras.utils.vis_utils import plot_model

from PIL import Image, ImageOps

import torch

from keras import models

import scipy.io

# from dfply import *

torch.manual_seed(4460)
np.random.seed(4460)

Using TensorFlow backend.


In [2]:
xtrain = scipy.io.loadmat('./data/xtrain.mat')
ytrain = scipy.io.loadmat('./data/ytrain.mat')

In [3]:
assert len(xtrain['TMRF_T2']) == len(ytrain['GS_T2'])
print('The sample size is', len(xtrain['TMRF_T2']))

The sample size is 20


In [4]:
def unet(pretrained_weights=None, input_size=(224, 224, 1)):
    inputs = Input(input_size)
    # conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    # conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    # pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    # delete the pool4 layer; instead, use a upsampling to get a up5 layer, to turn the unet
    up5 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(drop4))
    
    # the old merge 6
    merge5 = concatenate([drop3, up5], axis=3)
    # the old conv6
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    
    # to concat with conv2
    up6 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv5))
    merge6 = concatenate([conv2, up6])
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    

    conv10 = Conv2D(1, 1)(conv6)

    model = Model(input=inputs, output=conv10)
    model.compile(optimizer=Adam(lr=1e-4), loss='mse',  metrics=['mae', 'mse'])
    print(model.summary())

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model